In [ ]:
from typing import Optional, Callable, Tuple

import torch as th
from torch import nn
from torch.nn import functional as F
from torch.nn.utils import weight_norm

from transformer_utils import ApplyAttentionMask

class AttentionQKV(nn.Module):
    def __init__(self):
      super().__init__()
      self.apply_mask = ApplyAttentionMask()

    def forward(self, queries, keys, values, mask=None):

      # ...
      # https://arxiv.org/pdf/1706.03762.pdf

      key_dim = th.tensor(keys.shape[-1], dtype=th.float32)
      similarity = # ...

      masked_similarity = self.apply_mask(similarity, mask)
      weights = # ...
      output = # ...

      return output, attention_weights

class MultiHeadProjection(nn.Module):
    def __init__(self, n_heads, feature_sizes):
        super().__init__()
        self.attention_map = AttentionQKV()
        self.n_heads = n_heads

        for size in feature_sizes:
            assert size % self.n_heads == 0, 'Shape of feature input must be divisible by n_heads'

    def forward(self, inputs, mask=None):
      queries, keys, values = inputs
      queries_split = self._split_heads(queries)
      keys_split = self._split_heads(keys)
      values_split = self._split_heads(values)

      attention_output_split = self.attention_map(queries_split, keys_split, values_split, mask=mask)

      output = self._combine_heads(attention_output_split)

      return output

    def _split_heads(self, tensor):
      assert len(tensor.shape) == 3

      # ...

      batch_size, tensorlen = tensor.shape[0], tensor.shape[1]
      feature_size = tensor.shape[2]

      new_feature_size = # ...
      tensor = # ...

      return tensor

    def _combine_heads(self, tensor):
      assert len(tensor.shape) == 4

      # ...

      tensor =

      batch_size, tensorlen = tensor.shape[0], tensor.shape[1]
      feature_size = tensor.shape[-1]

      new_feature_size = # ...
      tensor = # ...

      return tensor

class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, input_shapes):
        super().__init__()
        self.qa_channels, self.ma_channels = input_shapes

        self.n_heads = n_heads
        self.attention_layer = MultiHeadProjection(n_heads, (self.qa_channels, self.ma_channels))

        assert self.qa_channels % self.n_heads == 0, and self.ma_channels % self.n_heads == 0 and \
                                            'Feature size must be divisible by n_heads'
        assert self.qa_channels == self.qa_channels and 'Cannot combine tensors with different shapes'

        self.query_layer = weight_norm(nn.Linear(self.qa_channels, self.qa_channels, bias = False))
        self.key_layer = weight_norm(nn.Linear(self.qa_channels, self.qa_channels, bias = False))
        self.value_layer = weight_norm(nn.Linear(self.ma_channels, self.ma_channels, bias = False))
        self.output_layer = weight_norm(nn.Linear(self.qa_channels, self.qa_channels, bias = False))

        def weights_init(m):
          nn.init.xavier_uniform_(m.weight.data)
        self.query_layer.apply(weights_init)
        self.key_layer.apply(weights_init)
        self.value_layer.apply(weights_init)
        self.output_layer.apply(weights_init)

      def forward(self, inputs, mask=None):
        assert (isinstance(inputs, tuple) or isinstance(inputs, list)) and len(inputs) == 2 and \
                                                        'Must pass query and memory'
        query_antecedent, memory_antecedent = inputs

        q = self.query_layer(query_antecedent)
        k = self.key_layer(memory_antecedent)
        v = self.value_layer(memory_antecedent)

        attention_output = self.attention_layer((q, k, v), mask=mask)
        output = self.output_layer(attention_output)

        return output

